### Data Cleaning

In [14]:
from sqlalchemy import create_engine
import pandas as pd
import re
from functools import reduce

def get_info():
    sqlitedb_path = '/home/juan/IronHack/ih_datamadpt0420_project_m1/data/raw_data_project_m1.db'
    engine = create_engine(f'sqlite:///{sqlitedb_path}')
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", engine)
    tables_lst = tables['name'].to_list()
    x = [pd.read_sql_query(f'select * from {i}', engine) for i in tables_lst]
    df_final = reduce(lambda left,right: pd.merge(left,right,on='uuid'), x)
    df_final.to_csv(f'/home/juan/IronHack/ih_datamadpt0420_project_m1/data/raw/data_base_2.csv')
    jobs_ids = list(df_final['normalized_job_code'].unique())
    return jobs_ids

In [15]:
get_info()



([None,
  '861a9b9151e11362eb3c77ca914172d0',
  '049a3f3a2b5f85cb2971ba77ad66e10c',
  'f4b2fb1aa40f661488e2782b6d57ad2f',
  '27af8700f5577cec835acee2cb90a2ff',
  'c1b670eba9ccb65e7c99f7da116d5b9c',
  'd71c4164c99a5f9bca773e755a72f40f',
  '2ee80cf30e2483dfe0947adc0428f8c5',
  '897ef18a9321616495831a4b4b0c1bb2',
  'a003702c0cea282e17967772dd83de5e',
  '04c82eae29e1c4a30eac0710b74ab276',
  'fcaf4aac737e53ad24b9f32f08b8740f',
  '3833bd699e7afe4665b595f282da0387',
  '19e384e977796c25780d8c28b12089c4',
  '474b962c13632a04e42a546be4f11dbb',
  'dac2e921c1f887d9efc4c06fa33c1397',
  '996da29c84f5eed00da40c230f4c4078',
  'cc2b26cc436bea36bda3bdd7367c411a',
  '25fe195cbf915c0824fa6c44e6d0008c',
  '76385cb24e831b3364279b9de5641d8f',
  '16b67d7f57698455aefbc1ae1b358b50',
  'b24ece3300d2f8ab34f9553e531183d6',
  '5db2c8d6539f983e1ca3d7c5b939e190',
  '36675c937946c8a4fc5de7d0b4c850de',
  '847165cfda6b1dc82ae22b967da8af2f',
  '2ca138b991c50dc6696238e5ad800142',
  '775190277a849cba701b306a7b374c0a',
  '0

In [177]:
# to drop na jobs
#def drop_na_jobs():
#    get_info = pd.read_csv(f'/home/juan/IronHack/ih_datamadpt0420_project_m1/data/raw/data_base.csv')
#    col_to_clean = 'normalized_job_code'
#    df_clean = get_info[get_info[col_to_clean].notna()]
#    return df_clean


In [185]:
#x = drop_na_jobs()
#x

In [182]:
#x = pd.unique(x['normalized_job_code'].to_list())
#j = [i for i in x]
#l = pd.DataFrame(j)
#l.to_csv(f'/home/juan/IronHack/ih_datamadpt0420_project_m1/data/raw/jobs_ids.csv')

In [67]:
#df_career['normalized_job_code'].to_list()

### TO CLEAN

To change:
    1. age 
    2. gender
    3. age_group (!)
    

In [134]:
from sqlalchemy import create_engine
import pandas as pd
import re
from functools import reduce
def get_all_data():
    df = pd.read_csv('/home/juan/IronHack/ih_datamadpt0420_project_m1/data/raw/all_data_merged.csv')
    return df

In [135]:
df = get_all_data()

In [142]:
### NAMES - 2

def clean_col(df):
    df.rename(columns={'English short name (using title case)': 'Country', 'title': 'Job Title', 'age': 'Age (years old)'}, inplace=True)
    df_2 = df[df['Job Title'].notna()]
    df_3 = df_2[['Country', 'Job Title', 'Age (years old)']]
    return df_3

In [143]:
clean_col(df)

,Country,Job Title,Age (years old)
1,Austria,Automatic Data Processing Planner,57
3,Austria,Data Coordinator,45
4,Austria,Database Developer,41
5,Austria,Data Entry Specialist,1990
7,Austria,Database Architect,47
...,...,...,...
9641,Slovakia,Database Management System Specialist (DBMS Sp...,2000
9644,Slovakia,Data Warehouse Developer,37
9645,Slovakia,Database Manager,53
9647,Slovakia,Data Officer,47


In [140]:
### AGE - 1
def get_age(df):
    df['age'] = df['age'].apply(lambda age: re.sub('[a-z]','',age))
    df = df.astype({'age':int})
    df['age'] = df['age'].apply(lambda age: age if age < 250 else 2016-age)
    return df

In [141]:
get_age(df)

,English short name (using title case),title,age
0,Austria,NaN,61
1,Austria,Automatic Data Processing Planner,57
2,Austria,NaN,32
3,Austria,Data Coordinator,45
4,Austria,Database Developer,41
...,...,...,...
9644,Slovakia,Data Warehouse Developer,37
9645,Slovakia,Database Manager,53
9646,Slovakia,NaN,24
9647,Slovakia,Data Officer,47


In [42]:
def clean_gender(gender):
    gender_low = gender.lower()
    if gender_low[0] is 'f':
        gender_f = re.sub(r'\b[f]\w+', 'female', gender_low)
        return gender_f
    if gender_low[0] is 'm':
        gender_m = re.sub(r'\b[N]\w+', 'male', gender_low)
        return gender_m
    else:
        return gender



In [187]:
def clean_children(children):
    children_low = children.lower()
    if children_low[0] is 'y':
        children_y = re.sub(r'\b[y]\w+', 'yes', children_low)
        return children_y
    if children_low[0] is 'n':
        children_n = re.sub(r'\b[n]\w+', 'no', children_low)
        return children_n
    else:
        children

df['dem_has_children'] = df['dem_has_children'].apply(clean_children)
df.head()

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,no,40_65,AT,countryside,no,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yes,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,no,26_39,AT,city,None,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,male,yes,40_65,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,female,yes,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
